### 1. Mounting Google Drive

  One advantage of using Google Colab is that connection with Google Drive is simple. By mounting Google Drive, the working files can be stored permanantly. After executing the following code block, log into the google account to finish the process.

In [1]:
from google.colab import drive
path='/content/gdrive/'
drive.mount(path)

Mounted at /content/gdrive/


In [2]:
import pandas as pd
import xarray 
import numpy as np
import matplotlib.pyplot as plt
import getpass
!pip install pydap
import pydap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 KB 14.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=b7629fe2ceaf23689aa3e8bd7229c399f6450cef0dbe5203c1d2cd485a026811
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [3]:
# To avoid warning messages
import warnings
warnings.filterwarnings('ignore')

In [4]:
ds_input = xarray.open_mfdataset('/content/gdrive/My Drive/Visiting_Demokritos/DO_ML_application/DATA/Input/all/*.nc', combine = 'by_coords')

In [5]:
ds_input

<xarray.Dataset>
Dimensions:       (time: 8217, lon: 69, lat: 48)
Coordinates:
  * time          (time) datetime64[ns] 1999-01-01T12:00:00 ... 2021-06-30T12...
  * lon           (lon) float32 12.12 12.17 12.21 12.25 ... 14.88 14.92 14.96
  * lat           (lat) float32 43.81 43.85 43.9 43.94 ... 45.69 45.73 45.77
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(8217, 48, 69), meta=np.ndarray>
    chl           (time, lat, lon) float32 dask.array<chunksize=(8217, 48, 69), meta=np.ndarray>
    no3           (time, lat, lon) float32 dask.array<chunksize=(8217, 48, 69), meta=np.ndarray>
    po4           (time, lat, lon) float32 dask.array<chunksize=(8217, 48, 69), meta=np.ndarray>
    o2            (time, lat, lon) float32 dask.array<chunksize=(8217, 48, 69), meta=np.ndarray>
    so            (time, lat, lon) float32 dask.array<chunksize=(8217, 48, 69), meta=np.ndarray>
Attributes: (12/57)
    CDI:                        Climate Data Interface version 1.9.9rc1 (http...
    history:                    Thu Feb 23 17:42:55 2023: cdo -remapbil,med-c...
    source:                     [1981-2016] ESA CCI SST v.2.0 L3C product (SS...
    institution:                GOS
    Conventions:                CF-1.4 
    title:                      Mediterranean Sea SST Analysis L4, Reprocesse...
    ...                         ...
    History:                    Translated to CF-1.0 Conventions by Netcdf-Ja...
    geospatial_lat_min:         43.81806182861328
    geospatial_lat_max:         45.77421188354492
    geospatial_lon_min:         12.152803421020508
    geospatial_lon_max:         14.955375671386719
    CDO:                        Climate Data Operators version 1.9.9rc1 (http...

In [6]:
# extend one dimension
ds_input=ds_input.expand_dims(dim={"var": 1},axis=-1)

In [ ]:
#dataset_no_time.to_netcdf('/content/gdrive/My Drive/Visiting_Demokritos/DO_ML_application/DATA/Input/all/preprocessed_data_for_CNN_16032023.nc')

# Open dataset ready for  CNN

In [7]:
dataset_time=ds_input

In [8]:
# numpy array of each variable
x1 = dataset_time['analysed_sst'].values  # shape (time, height, width, channels)
x2 = dataset_time['chl'].values  # shape (time, height, width, channels)
x3 = dataset_time['no3'].values  # shape (time, height, width, channels)
x4 = dataset_time['po4'].values  # shape (time, height, width, channels)
x5 = dataset_time['so'].values  # shape (time, height, width, channels)


In [ ]:
x1

In [22]:
# CALCULATE THE MEAN FOR EACH VARIABLE
x1_mean = np.nanmean(x1)
x2_mean = np.nanmean(x2)
x3_mean = np.nanmean(x3)
x4_mean = np.nanmean(x4)
x5_mean = np.nanmean(x5)
print(x1_mean)
print(x2_mean)
print(x3_mean)
print(x4_mean)

290.79086
0.3208867
4.4299054
0.02539813


In [23]:
# fill nan with the mean of each variable
x1_new=np.nan_to_num(x1,nan=x1_mean)
x2_new=np.nan_to_num(x2,nan=x2_mean)
x3_new=np.nan_to_num(x3,nan=x3_mean)
x4_new=np.nan_to_num(x4,nan=x4_mean)
x5_new=np.nan_to_num(x5,nan=x5_mean)

In [ ]:
x.shape

(8217, 48, 69, 5)

In [24]:
# concatenate all the input variables
x = np.concatenate([x1_new, x2_new, x3_new, x4_new, x5_new], axis=-1)  # concatenate along the

In [ ]:
# output variable = o2
y_old =dataset_time['o2'].values
y_old

In [26]:
y_mean = np.nanmean(y_old)

In [27]:
# fill the nan with the o2 mean value
y=np.nan_to_num(y_old,nan=y_mean)

# ▶ Image regression using CNN

To perform time-series prediction using a Convolutional Neural Network (CNN) on NetCDF files in Python, you would need to follow the steps outlined below:

1. Load the NetCDF data: You can use the xarray library to load the data from the NetCDF files into a DataArray object.

2. Preprocess the data: This may include normalizing the data, splitting the data into training and testing sets, and transforming the data into a format suitable for training the CNN.

3. Define the CNN architecture: You can use the Keras or TensorFlow libraries to define the architecture of your CNN. You'll need to specify the number of layers, the type of layers (e.g., Conv2D, MaxPooling2D, etc.), and the number of filters for each layer.

4. Train the model: You'll use the training data to train the model. You'll also specify the number of epochs, the batch size, and the optimizer.

5. Evaluate the model: You'll use the testing data to evaluate the performance of the model. You can use metrics such as mean squared error (MSE) or mean absolute error (MAE) to evaluate the model's performance.

6. Make predictions: Finally, you can use the trained model to make predictions on new time-series data.

In [28]:
# importing os module 
import os
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import h5py

https://github.com/pangeo-data/WeatherBench/blob/master/notebooks/3-cnn-example.ipynb 

In [29]:
# Depending on your combination of package versions, this can raise a lot of TF warnings... 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import seaborn as sns
import pickle
#from src.score import *
from collections import OrderedDict
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, MaxPooling2D, Flatten

In [31]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
y_train.size

In [41]:
#Normalize the data
pixel_means = x_train.mean(axis = 0)
pixel_stds = x_test.std(axis = 0)

x_train = (x_train - pixel_means) / pixel_stds
x_test = (x_test - pixel_means) / pixel_stds 

### define the CNN architecture.

In [33]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=[3, 3], activation='relu', input_shape=[48, 69, 5]),
    #tf.keras.layers.MaxPooling2D(pool_size=[2,2]),
    #tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu'),
    #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 67, 32)        1472      
                                                                 
 flatten (Flatten)           (None, 98624)             0         
                                                                 
 dense (Dense)               (None, 1)                 98625     
                                                                 
Total params: 100,097
Trainable params: 100,097
Non-trainable params: 0
_________________________________________________________________


In [42]:
x_train.shape

(6573, 48, 69, 5)

In [43]:
y_train.shape

(6573, 48, 69, 1)

In [44]:
# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError()])


In [45]:
#train the model on our labeled dataset. Here, X_train is a 3D array of shape (38, 69, 5) containing the input image sequences, 
#and y_train is a 1D array of shape (38, 69, 1)
#containing the corresponding output variables.
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
#use our trained model to predict the output variable for new image sequences. 
#Here, X_test is a 4D array of shape (num_samples, height, width, channels) containing the test image sequences.
y_pred = model.predict(X_test)


----